## Hosting Capacity com circuitos IEEE
_Autor: Leonardo Jaime_

### Imports

In [100]:
# !pip install py-dss-interface

# instalação da biblioteca do OpenDSS

# as outras bibliotecas estão inclusas no anaconda

In [101]:
# Importação das bibliotecas utilizadas
import py_dss_interface
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import random
import functions
from bokeh.io import show, curdoc
from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_file
from bokeh.models import Range1d
from bokeh.themes import built_in_themes
from itertools import permutations

Repósitorio GitHub: <a href="https://github.com/leojms/OpenDSS">Repositório</a> 

### Funções

##### Variáveis

In [102]:
# usuario utilizado (esse método só funciona comigo mesmo kkkk)
# caso use em outro computador, apenas substituir o diretorio por endereços da sua maquina
u_senai = "leonardo.simoes"
u_note = "leona"
# Diretório do arquivo do alimentador
# caso seja utilizado um outro alimentador, ou em outra máquina, é necessário criar a variável com o novo diretório
diretorio = fr'C:\Users\{u_senai}\OneDrive - Sistema FIEB\centro_comp\host_cap'

In [103]:
# variaveis gerais
#objeto que executa o OpenDSS
dss = py_dss_interface.DSSDLL()
#fator de potencia
pf=1
#relação entre p e s
kva_to_kw = 1
#valor para padronizar a escolha randomica das barras
seed_value = 1685

OpenDSS Started successfully! 
OpenDSS Version 9.4.0.1 (64-bit build); License Status: Open 




##### Preparação do circuito

In [104]:
#criação do sistema fv
def fv1(kva, bus, kv, pmpp):
    #condicional caso nao haja sistema fotovoltaico
    if kva == 0 or pmpp == 0:
        return
    else:
        # criação das curvas do sistema fotovoltaico
        # curva do comportamento do fator de temperatura de acordo com o aumento da temperatura
        dss.text("new XYcurve.ctemp npts=4 xarray=[0 25 75 100] yarray=[1.2 1.0 0.8 0.6]")
        # Curva de eficiência do inversor
        dss.text("new XYcurve.ceficiencia npts=5 xarray=[0.1 0.2 0.4 1 2] yarray=[0.86 0.9 0.93 0.97 0.8]")
        # Curva de irradiancia diaria baseada na irradiancia solar local
        dss.text("new loadshape.cirrad npts=24 interval=1")
        dss.text(fr"~ mult=(file=[{diretorio}\curvas\curva_fv.txt])")
        # curva da variação de temperatura da placa fotovoltaica diaria
        dss.text("new tshape.t npts=24 interval=1")
        dss.text(fr"~ temp=(file=[{diretorio}\curvas\curva_temp.txt])")
        # criação do sistema fotovoltaico
        # criação do objeto e associação das curvas criadas
        dss.text(f"new PVSystem.{bus} phases=3 bus1={bus} kv={kv} irrad=0.98 pmpp={pmpp} kva={kva} temperature=25 pf=1")
        dss.text("~ %cutin=0.1 %cutout=0.1 effcurve=ceficiencia P-tCurve=ctemp Daily=cirrad Tdaily=t")


#função para zerar resistencias na entrada da rede
def zera_reatancia(sep):
    if sep == 13:
        dss.text(fr"edit TRANSFORMER.SUB %loadloss=0.000001 xhl=0.0000001")
        dss.text(fr"edit Transformer.REG1 %loadloss=0.000001 xhl=0.0000001")
        dss.text(fr"edit TRANSFORMER.REG2 %loadloss=0.000001 xhl=0.0000001")
        dss.text(fr"edit TRANSFORMER.REG3 %loadloss=0.000001 xhl=0.0000001")
    elif sep == 34:
        dss.text(fr"edit TRANSFORMER.SUBXF %loadloss=0.000001 xhl=0.0000001")
    elif sep ==5:
        dss.text("edit Reactor.MDV_SUB_1_HSB x=0.0000001 r=0.0000001")
        dss.text(r"edit Transformer.MDV_SUB_1 %loadloss=0.000001 xhl=0.0000001")
    else:
        pass


def reg(sep):
    if sep == 13:
        dss.text(fr"edit regcontrol.Reg1 vreg=122")
        dss.text(fr"edit regcontrol.Reg2 vreg=122")
        dss.text(fr"edit regcontrol.Reg3 vreg=122")
    elif sep == 34:
        dss.text(fr"edit regcontrol.cReg1a vreg=30")
        dss.text(fr"edit regcontrol.cReg1b vreg=30")
        dss.text(fr"edit regcontrol.cReg1c vreg=30")


#função para selecionar as barras que são trifásicas no sistema,
#a fim de serem utilizadas para aplicar o fv
def barras_trifasicas(qtd_barras):
    flag = 1
    barras_hc_geral = list()
    barras_kv = dict()
    barras = dss.circuit_all_bus_names()
    #loop para mapear as barras
    for barra in barras:
        #setar a barra que vai ser avaliada
        dss.circuit_set_active_bus(barra)
        #mapeando a tensão da barra
        barra_kv = dss.bus_kv_base()
        #avaliando o número de fases
        num_fases = len(dss.bus_nodes())
        #condicional para avaliar se são trifásicas e de média tensão
        if num_fases == 3 and barra!= 'sourcebus' and len(dss.bus_load_list()) != 0:
            barras_hc_geral.append(barra)
            barras_kv[barra] = barra_kv
    #quantidade percentual de barras utilizadas que serão utilizadas
    if qtd_barras <= len(barras_hc_geral):
        barras_hc = list(permutations(barras_hc_geral, qtd_barras))
    else:
        barras_hc = 0
        flag = 0
    return barras_hc, barras_kv, flag

##### Hosting capacity

In [105]:
#função para executar o algoritmo de hosting capacity
def hc(arquivo, sep, kw_step, load_mult, thevenin_pu):
    #criação de listas para armazenar os valores
    fv_kva = list()
    fv_kw = list()
    fv_kvar = list()
    thev_kw_com_fv = list()
    thev_kvar_com_fv = list()
    tensao_min = list()
    tensao_max = list()
    sobretensao_lista = list()
    sobrecarga_lista = list()
    i_list = list()
    barras_fv = list()
    q_barras = list()
    perdas_kw = list()
    perdas_kvar = list()
    fv_kw_unit = list()
    fv_kva_unit = list()
    barras_com_sobretensao = list()
    linhas_com_sobrecarga = list()
    lista = list()
    qtd_barras = 0
    flag = 1
    fv_kva_final = -2
    #loop para realizar as simulações variando as porcentagens de utilização das barras
    while flag == 1:
        # limpar o buffer
        dss.text("clear")
        # compilar o arquivo que contém o alimentador
        dss.text(f"compile [{arquivo}]")
        #chamar a função que zera a reatância
        zera_reatancia(sep)
        reg(sep)
        #setar um redutor de carga, para simular um pior cenário
        dss.text(f"set loadmult = {load_mult}")
        #setar um valor para tensão do equivalente de thevenin, a fim de padronizar a entrada
        #dss.text(f"edit Vsource.SOURCE pu={thevenin_pu}")
        #configuração de simulação e resolução do fator de potência
        dss.text("set Maxiterations=20")
        dss.solution_solve()
        #selecionar as barras que vão ser utilizadas para a implementação do fv
        barras_hc, flag = barras_trifasicas(qtd_barras)[0::2]
        if flag == 0:
                break
        print(barras_hc[qtd_barras])
        len_barras_hc_tupla = len(list(barras_hc[0]))
        if len(barras_hc) == 1:
            len_barras_hc = 1
        else:
            len_barras_hc = len(barras_hc) - 1
        fv_kva_final = -2
        for i in range(0, len_barras_hc):
            # limpar o buffer
            dss.text("clear")
            # compilar o arquivo que contém o alimentador
            dss.text(f"compile [{arquivo}]")
            #chamar a função que zera a reatância
            zera_reatancia(sep)
            reg(sep)
            #setar um redutor de carga, para simular um pior cenário
            dss.text(f"set loadmult = {load_mult}")
            #setar um valor para tensão do equivalente de thevenin, a fim de padronizar a entrada
            #dss.text(f"edit Vsource.SOURCE pu={thevenin_pu}")
            #configuração de simulação e resolução do fator de potência
            dss.text("set Maxiterations=20")
            dss.solution_solve()
            #criar uma lista para armazenar essas barras
            barras_fv_atual = list()
            #varrer todas as barras que serão utilizadas e aplicar o fv
            for barra in barras_hc[i]:
                #criar o fv
                fv1(kva=kw_step * kva_to_kw, bus=barra, kv=barras_trifasicas(qtd_barras)[1][barra] * math.sqrt(3), pmpp=kw_step)
                #functions.define_3ph_pvsystem(dss=dss, bus=barra, kv=barras_trifasicas(0.3)[1][barra], kva=kw_step * kva_to_kw, pmpp = kw_step)
                #adicionar um marcador, para identificar a localização
                functions.add_bus_marker(dss=dss, bus=barra, color="red", size_marker=2)
                #inserir o nome das barras na lista com os nomes
                barras_fv_atual.append(barra)
            #identificar a quantidade de barras que foram colocados os fv
            #calculo do fluxo de potencia
            dss.text("interpolate")
            dss.text("set Maxiterations=20")
            dss.solution_solve()
            #caso seja necessário, plot do circuito e do perfil de tensao (apenas descomentar)
            #IMPORTANTE: caso esteja no loop irá plotar um circuito para cada percentual simulado
            #dss.text("plot circuit Power max=2000 y y C1=$00FF0000")
            #dss.text("Plot Profile Phases = All")
            #flags para indicar se há violação na rede
            sobretensao = False
            sobrecarga = False
            #limite de tensao avaliado
            v_limite = 1.05
            #numero de iterações
            i=0
            #loop de cálculo dos indicadores de HC
            while not sobretensao and not sobrecarga and i<10000:
                if qtd_barras == 0:
                    break
                else:
                    pass
                #incremento das iterações
                i += 1
                #aumento do sistema fotovoltaico de acordo com o passo definido
                functions.increment_pv_size(dss=dss, p_step=kw_step, kva_to_kw=kva_to_kw, pf=pf, i=i)
                #calculo do fluxo
                dss.text("set Maxiterations=20")
                dss.solution_solve()
                #armazenamento das tensoes
                tensoes = dss.circuit_all_bus_vmag_pu()
                #avaliação do ponto que possui a maior tensão
                v_max = max(tensoes)
                
                #condicional para avaliar a existência de sobretensão
                if v_max > v_limite:
                    sobretensao = True
                    v_barra_max = 0
                    barra_max = '0'
                    
                    for barra in dss.circuit_all_bus_names():
                        dss.circuit_set_active_bus(barra)
                        v_barra = max(dss.bus_pu_vmag_angle()[0:6:2])
                        if v_barra > v_barra_max:
                            v_barra_max = v_barra
                            barra_max = barra  
                    barra_com_sobretensao = barra_max 
                        
                #lógica para avaliar sobrecarga 
                #Selecionada a primeira linha do circuito
                dss.lines_first()
                #loop para varrer todas as linhas do circuito
                for _ in range(dss.lines_count()):
                    #Selecionar a linha que vai ser avaliada
                    dss.circuit_set_active_element(f"line.{dss.lines_read_name()}")
                    #armazenar a corrente atual da linha
                    i_linha = dss.cktelement_currents_mag_ang()
                    #armazenar a corrente nominal da linha
                    i_nom_linha = dss.lines_read_norm_amps()
                    
                    #avaliar se a corrente atual é maior que a corrente nominal
                    #se for, há sobrecarga
                    if (max(i_linha[0:12:2]) / i_nom_linha) > 1:
                        sobrecarga = True
                        linha_com_sobrecarga = dss.lines_read_name()
                        break
                
                    #ir para a próxima linha    
                    dss.lines_next()
            if sobrecarga == False:
                linha_com_sobrecarga = 0
            if sobretensao == False:
                barra_com_sobretensao = 0
                
            #Selecionar o valor anterior do fotovoltaico, para não ter violação
            functions.increment_pv_size(dss=dss, p_step=kw_step, kva_to_kw=kva_to_kw, pf=pf, i=i-1)
            #calculo do fluxo
            dss.text("set Maxiterations=20")
            dss.solution_solve()
            #organização dos valores obtidos
            
            fv_kva_pad = (i-1) * len(barras_hc[qtd_barras]) * kw_step
            
            if fv_kva_pad > fv_kva_final:
                try:
                    fv_kw_unit_elem = (functions.get_total_pv_powers(dss)[0])/(len_barras_hc_tupla)
                except ZeroDivisionError:
                    fv_kw_unit_elem = 0
                
                fv_kva_unit_elem = (i-1) * kw_step
                if fv_kva_unit_elem < 0:
                    fv_kva_unit_elem = 0
                else:
                    pass
                fv_kva_final = fv_kva_pad
                fv_kva_unit_elem_final = fv_kva_unit_elem
                fv_kw_unit_elem_final = fv_kw_unit_elem
                perdas_kw_final = dss.circuit_losses()[0] / 10**3
                perdas_kvar_final = dss.circuit_losses()[1] / 10**3
                fv_kw_final, fv_kvar_final = functions.get_total_pv_powers(dss)[0:2]
                thev_kw_com_fv_final = -1 * dss.circuit_total_power()[0]
                thev_kvar_com_fv_final = -1 * dss.circuit_total_power()[1]
                v_pu = dss.circuit_all_bus_vmag_pu()
                tensao_min_final = min(v_pu)
                tensao_max_final = max(v_pu)
                sobretensao_final = sobretensao
                sobrecarga_final = sobrecarga
                i_final = i
                barras_fv_atual_final = barras_fv_atual
                qtd_barras_final = int(qtd_barras)
                barra_com_sobretensao_final = barra_com_sobretensao
                linha_com_sobrecarga_final = linha_com_sobrecarga
                
        fv_kva_unit.append(fv_kva_unit_elem_final)
        fv_kw_unit.append(fv_kw_unit_elem_final) 
        perdas_kw.append(perdas_kw_final)
        perdas_kvar.append(perdas_kvar_final)
        fv_kva.append(fv_kva_final)
        fv_kw.append(fv_kw_final)
        fv_kvar.append(fv_kvar_final)
        thev_kw_com_fv.append(thev_kw_com_fv_final)
        thev_kvar_com_fv.append(thev_kvar_com_fv_final)
        tensao_min.append(tensao_min_final)
        tensao_max.append(tensao_max_final)
        sobretensao_lista.append(sobretensao_final)
        sobrecarga_lista.append(sobrecarga_final)
        i_list.append(i_final)
        barras_fv.append(barras_fv_atual_final)
        q_barras.append(int(qtd_barras_final))
        barras_com_sobretensao.append(barra_com_sobretensao_final)
        linhas_com_sobrecarga.append(linha_com_sobrecarga_final)
        qtd_barras = qtd_barras + 1
    
    #criação de um dicionário e um dataframe, para exportar os dados para excel    
    dicio = dict()
    dicio["quantidade de barras utilizadas"] = q_barras
    dicio["potencia fv por unidade (kVA)"] = fv_kva_unit
    dicio["potencia fv por unidade (kW)"] = fv_kw_unit
    dicio["potencia fv (kVA)"] = fv_kva
    dicio["potencia fv (kW)"] = fv_kw
    dicio["potencia fv (kVAr)"] = fv_kvar
    dicio["Perdas (kW)"] = perdas_kw
    dicio["Perdas (kVAr)"] = perdas_kvar
    dicio["potencia subestacao (kW)"] = thev_kw_com_fv
    dicio["potencia subestacao (kVAr)"] = thev_kvar_com_fv
    dicio["tensao minima"] = tensao_min
    dicio["tensao maxima"] = tensao_max
    dicio["sobretensao"] = sobretensao_lista
    dicio["sobrecarga"] = sobrecarga_lista
    dicio["Barra com sobretensão"] = barras_com_sobretensao
    dicio["numero de iteracoes"] = i_list
    dicio["Linha com sobrecarga"] = linhas_com_sobrecarga
    dicio["barras com fotovoltaico inserido"] = barras_fv
    df = pd.DataFrame.from_dict(dicio)
    if sep == 5:
        nome = f'Dados HC ckt{sep}.xlsx'
    elif sep == 8500:
        nome = f'Dados HC {sep}-node.xlsx'
    else:
        nome = f'Dados HC {sep} barras.xlsx'
    df.to_excel(fr'{diretorio}\{nome}', index=False)
    print(f"O arquivo de dados {nome} foi salvo em excel com sucesso")
    return df

### Calcular Fluxo

In [106]:
#########################
# Variáveis de cada SEP #
#########################

# 13 barras
local_13 = fr"{diretorio}\IEEE\13Bus"
nome_arquivo_13 = "IEEE13Nodeckt.dss"
arquivo_13 = fr"{local_13}\{nome_arquivo_13}"
load_mult_13 = 0.2
thevenin_pu_13 = 1.045
kw_step_13 = 10
#linha_13 = "line.650632"

#34 barras
local_34 = fr"{diretorio}\IEEE\34Bus"
nome_arquivo_34 = "ieee34Mod2.dss"
arquivo_34 = fr"{local_34}\{nome_arquivo_34}"
load_mult_34 = 0.9
thevenin_pu_34 = 1.03
kw_step_34 = 100
#linha_34 = "line.L1"

#CKT 5
local_5 = fr"{diretorio}\circuito\ckt5"
nome_arquivo_5 = "Master_ckt5.dss"
arquivo_5 = fr"{local_5}\{nome_arquivo_5}"
load_mult_5 = 0.2
thevenin_pu_5 = 1.045
kw_step_5 = 5

#123 barras
local_123 = fr"{diretorio}\IEEE\123Bus"
nome_arquivo_123 = "IEEE123Master.dss"
arquivo_123 = fr"{local_123}\{nome_arquivo_123}"
load_mult_123 = 0.2
thevenin_pu_123 = 1.045
kw_step_123 = 1

#8500 node
local_8500 = fr"{diretorio}\IEEE\8500-Node"
nome_arquivo_8500 = "Master.dss"
arquivo_8500 = fr"{local_8500}\{nome_arquivo_8500}"
load_mult_8500 = 0.2
thevenin_pu_8500 = 1.045
kw_step_8500 = 10


#função para rodar a simulação, indicando as variáveis do sep preferido para o cálculo do HC
def simular(sep):
    if sep == 13:
        arquivo = arquivo_13
        load_mult = load_mult_13
        thevenin_pu = thevenin_pu_13
        kw_step = kw_step_13
    elif sep == 34:
        arquivo = arquivo_34
        load_mult = load_mult_34
        thevenin_pu = thevenin_pu_34
        kw_step = kw_step_34
    elif sep == 123:
        arquivo = arquivo_123
        load_mult = load_mult_123
        thevenin_pu = thevenin_pu_123
        kw_step = kw_step_123
    elif sep == 5:
        arquivo = arquivo_5
        load_mult = load_mult_5
        thevenin_pu = thevenin_pu_5
        kw_step = kw_step_5
    elif sep == 8500:
        arquivo = arquivo_8500
        load_mult = load_mult_8500
        thevenin_pu = thevenin_pu_8500
        kw_step = kw_step_8500
    else:
        print("O número referente ao SEP está incorreto")
        return
    df = hc(arquivo=arquivo, sep=sep, kw_step=kw_step, load_mult=load_mult, thevenin_pu=thevenin_pu)
    return df

#função para gerar os gráficos
def plot(sep):
    df = simular(sep)
    if sep == 5:
        nome = f'HC ckt{sep}'
    elif sep == 8500:
        nome = f'HC {sep}-node'
    else:
        nome = f'HC {sep} barras'
    output_file(fr'{diretorio}\{nome} bar.html')
    curdoc().theme = 'dark_minimal'
    potencia_fv = figure(x_axis_label="Quantidade de Barras com FV", title="Potência FV por unidade (kW)")
    potencia_fv.vbar(x=df["quantidade de barras utilizadas"], top=df["potencia fv por unidade (kW)"], color='white', width=0.5)
    potencia_fv_unit = figure(x_axis_label="Quantidade de Barras com FV", title="Potência FV (kW)")
    potencia_fv_unit.vbar(x=df["quantidade de barras utilizadas"], top=df["potencia fv (kW)"], color='aquamarine', width=0.5)
    potencia_sub = figure(x_axis_label="Quantidade de Barras com FV", title="Potência subestação (kW)")
    potencia_sub.vbar(x=df["quantidade de barras utilizadas"], top=df["potencia subestacao (kW)"], color='hotpink', width=0.5)
    perdas = figure(x_axis_label="Quantidade de Barras com FV", title="Perdas (kW)")
    perdas.vbar(x=df["quantidade de barras utilizadas"], top=df["Perdas (kW)"], color='lawngreen', width=0.5)
    grid_layout_bar = gridplot([[potencia_fv, potencia_fv_unit], [potencia_sub, perdas]])
    show(grid_layout_bar)
    output_file(fr'{diretorio}\{nome}.html')
    curdoc().theme = 'dark_minimal'
    potencia_fv = figure(x_axis_label="Quantidade de Barras com FV", title="Potência FV por unidade (kW)")
    potencia_fv.line(x=df["quantidade de barras utilizadas"], y=df["potencia fv por unidade (kW)"], color='white')
    potencia_fv_unit = figure(x_axis_label="Quantidade de Barras com FV", title="Potência FV (kW)")
    potencia_fv_unit.line(x=df["quantidade de barras utilizadas"], y=df["potencia fv (kW)"], color='aquamarine')
    potencia_sub = figure(x_axis_label="Quantidade de Barras com FV", title="Potência subestação (kW)")
    potencia_sub.line(x=df["quantidade de barras utilizadas"], y=df["potencia subestacao (kW)"], color='hotpink')
    perdas = figure(x_axis_label="Quantidade de Barras com FV", title="Perdas (kW)")
    perdas.line(x=df["quantidade de barras utilizadas"], y=df["Perdas (kW)"], color='lawngreen')
    grid_layout = gridplot([[potencia_fv, potencia_fv_unit], [potencia_sub, perdas]])
    show(grid_layout)
    

In [107]:
#pode ser colocados valores 123, 8500 e ckt 5 (5)
seps = [13, 34]


for sep in seps:
    plot(sep)

dss.text("plot circuit Power max=2000 y y C1=$00FF0000")


()
('671',)
('634', '675')
('634', '692', '671')
('634', '671', '670', '692')
('634', '671', '670', '675', '692')
O arquivo de dados Dados HC 13 barras.xlsx foi salvo em excel com sucesso
()
('mid824',)
('mid806', 'mid830')
('mid806', 'mid824', 'mid858')
('mid806', 'mid824', 'mid828', 'mid840')


In [ ]:
buses = dss.circuit_all_bus_names()
selected_buses = list()

for bus in buses:
    dss.circuit_set_active_bus(bus)
    num_phases = len(dss.bus_nodes())
    bus_kv = dss.bus_kv_base()
    if len(dss.bus_load_list()) != 0 and num_phases == 3 and bus != 'sourcebus':
        selected_buses.append(bus)
    else:
        pass

selected_buses


['634', '671', '692', '675', '670']

In [ ]:
dss.circuit_set_active_bus('800')
dss.bus_pu_vmag_angle()[0:6:2]

[0.0]